In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'f:\\end_to_end_flight_price_prediction\\FLIGHT_PRICE_PREDICTION'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [5]:
from Flight_Price.constants.__init__ import *
from Flight_Price.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [ ]:
import os
from Flight_Price.logging import logger


In [8]:
import pandas as pd

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            for col in all_cols:
                # Check if column is in the schema
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

                # Layer to validate if the datatype is 'object'
                is_object = data[col].dtype == 'object'
                with open(self.config.STATUS_FILE, 'a') as f:
                    f.write(f"Column: {col} , Is Object: {is_object}\n")

            return validation_status

        except Exception as e:
            raise e


In [9]:
try:
    config = ConfigurationManager()
    print("ConfigurationManager initialized.")
    data_validation_config = config.get_data_validation_config()
    print(f"Data Validation Config: {data_validation_config}")
    data_validation = DataValidation(config=data_validation_config)
    print("DataValidation initialized.")
    data_validation.validate_all_columns()
    print("Validation completed successfully.")
except Exception as e:
    print(f"An error occurred: {str(e)}")
    raise e


[2024-12-01 23:27:12,364: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-01 23:27:12,367: INFO: common: yaml file: Params.yaml loaded successfully]
[2024-12-01 23:27:12,371: INFO: common: yaml file: Schema.yaml loaded successfully]
[2024-12-01 23:27:12,374: INFO: common: created directory at: artifacts]
ConfigurationManager initialized.
[2024-12-01 23:27:12,377: INFO: common: created directory at: artifacts/data_validation]
Data Validation Config: DataValidationConfig(root_dir='artifacts/data_validation', STATUS_FILE='artifacts/data_validation/status.txt', unzip_data_dir='artifacts/data_ingestion/Cleaned_Data.csv', all_schema=ConfigBox({'Airline': 'int64', 'Destination': 'int64', 'Duration': 'int64', 'Total_Stops': 'int64', 'Price': 'float64', 'Date_of_Journey_day': 'int64', 'Date_of_Journey_month': 'int64', 'Dep_Time_hour': 'int64', 'Dep_Time_minute': 'int64', 'Arrival_Time_hour': 'int64', 'Arrival_Time_minute': 'int64', 'Duration_hour': 'int64', 'Duration_m